In [ ]:
import os
from bs4 import BeautifulSoup
import re
import subprocess
import pandas as pd
schools = ['youngstown-state','yale','xavier','wyoming','wright-state','wofford','wisconsin','winthrop','william-mary','wichita-state','western-michigan','western-kentucky','western-illinois','western-carolina','west-virginia','weber-state','washington-state','washington','wake-forest','wagner','virginia-tech','virginia-military-institute','virginia-commonwealth','virginia','villanova','vermont','vanderbilt','valparaiso','utah-valley','utah','utah-state','california','ucla','california-santa-barbara','california-riverside','california-irvine','california-davis','tulsa','tulane','troy','towson','toledo','texas-san-antonio','texas-pan-american','texas-el-paso','texas-arlington','texas-tech','texas-state','texas-southern','texas','texas-christian','texas-am-corpus-christi','texas-am','tennessee-martin','tennessee','tennessee-tech','tennessee-state','temple','syracuse','stony-brook','stetson','stephen-f-austin','stanford','st-johns-ny','st-francis-ny','st-bonaventure','southern-utah','southern-mississippi','southern-methodist','southern','southern-illinois-edwardsville','southern-illinois','southern-california','southeastern-louisiana','southeast-missouri-state','south-florida','south-dakota-state','south-dakota','south-carolina-upstate','south-carolina-state','south-carolina','south-alabama','siena','seton-hall','seattle','savannah-state','santa-clara','san-jose-state','san-francisco','san-diego','san-diego-state','samford','sam-houston-state','saint-peters','saint-marys-ca','saint-louis','saint-josephs','saint-francis-pa','sacred-heart','sacramento-state','rutgers','robert-morris','rider','richmond','rice','rhode-island','radford','quinnipiac','ipfw','purdue','providence','princeton','presbyterian','prairie-view','portland-state','portland','pittsburgh','pepperdine','pennsylvania','penn-state','pacific','oregon-state','oregon','oral-roberts','nebraska-omaha','old-dominion','oklahoma-state','oklahoma','ohio-state','ohio','oakland','notre-dame','northwestern','northwestern-state','northern-kentucky','northern-iowa','northern-illinois','northern-colorado','northern-arizona','northeastern','north-texas','north-florida','north-dakota-state','north-dakota','north-carolina-wilmington','north-carolina-greensboro','north-carolina-asheville','north-carolina','north-carolina-state','north-carolina-central','north-carolina-at','north-alabama','norfolk-state','njit','nicholls-state','niagara','new-orleans','new-mexico-state','new-mexico','new-hampshire','nevada-las-vegas','nevada','nebraska','navy','murray-state','mount-st-marys','morgan-state','morehead-state','montana-state','montana','monmouth','missouri-kansas-city','missouri','missouri-state','mississippi-valley-state','mississippi-state','mississippi','minnesota','milwaukee','middle-tennessee','michigan','michigan-state','miami-oh','miami-fl','mercer','memphis','mcneese-state','massachusetts-lowell','massachusetts','maryland-eastern-shore','maryland-baltimore-county','maryland','marshall','marquette','marist','manhattan','maine','loyola-marymount','loyola-md','loyola-il','louisville','louisiana-monroe','louisiana-tech','louisiana-state','louisiana-lafayette','longwood','long-island-university','long-beach-state','arkansas-little-rock','lipscomb','liberty','lehigh','lamar','lafayette','la-salle','kentucky','kent-state','kennesaw-state','kansas-state','kansas','james-madison','jacksonville-state','jacksonville','jackson-state','iupui','iowa-state','iowa','iona','indiana-state','indiana','incarnate-word','illinois-chicago','illinois-state','illinois','idaho','idaho-state','howard','houston','houston-baptist','holy-cross','hofstra','high-point','hawaii','harvard','hartford','hampton','green-bay','grand-canyon','grambling','gonzaga','georgia-tech','georgia-state','georgia-southern','georgia','georgetown','george-washington','george-mason','gardner-webb','furman','fresno-state','fordham','florida-state','florida-international','florida-gulf-coast','florida','florida-atlantic','florida-am','fairleigh-dickinson','fairfield','evansville','elon','eastern-washington','eastern-michigan','eastern-kentucky','eastern-illinois','east-tennessee-state','east-carolina','duquesne','duke','drexel','drake','detroit-mercy','depaul','denver','delaware-state','delaware','dayton','davidson','dartmouth','creighton','cornell','coppin-state','connecticut','columbia','colorado-state','colorado','college-of-charleston','colgate','coastal-carolina','cleveland-state','clemson','citadel','cincinnati','chicago-state','chattanooga','charlotte','charleston-southern','central-michigan','central-florida','central-connecticut-state','central-arkansas','canisius','campbell','california-baptist','cal-state-northridge','cal-state-fullerton','cal-state-bakersfield','cal-poly','butler','buffalo','bucknell','bryant','brown','brigham-young','bradley','bowling-green-state','boston-university','boston-college','boise-state','binghamton','bethune-cookman','belmont','baylor','ball-state','austin-peay','auburn','army','arkansas-pine-bluff','arkansas-state','arkansas','arizona','arizona-state','appalachian-state','american','alcorn-state','albany-ny','alabama-birmingham','alabama-state','alabama','alabama-am','akron','air-force','abilene-christian']

In [ ]:
folder = 'CBB'
year = 2019

try:
    os.mkdir(folder)
    print("Directory {} created".format(folder)) 
except FileExistsError:
        pass
tables = {}
for team in schools:
    target = os.path.join(folder,team + str(year) + '.html')

    # get the files
    if not os.path.exists(target):
        subprocess.call(['wget','-O',target,
        'https://www.sports-reference.com/cbb/schools/{}/{}.html'.format(team,year)])
        fs = os.path.getsize(target)
        if fs < 10:
            os.remove(target)
            continue
    
    # load the data
    tables[team] = {}

    with open(target,'rt', encoding = "ISO-8859-1") as fp:
        data = fp.read()
    tables[team]['name'] = re.findall('2018-19 (.*) Roster and Stats',data)[0]
    bs = BeautifulSoup(data,features="lxml")
    tables[team]['logo'] = re.findall('(http.*png)',str(bs.find_all('img',{"class": "teamlogo"})[0]))[0]
    tables[team]['conf'] = re.findall('/cbb/conferences/(.*)/{}.html'.format(year),data)[0]

    # collect all the tables
    m = re.findall(r'<!--[ \n]*(<div[\s\S\r]+?</div>)[ \n]*-->',data)
    m2 = re.findall(r'(<div class="table_outer_container">[ \n]*<div class="overthrow table_container" id="div_roster">[\s\S\r]+?</table>[ \n]*</div>[ \n]*</div>)',data)
    m4 = re.findall(r'(<div class="table_outer_container">[ \n]*<div class="overthrow table_container" id=".*">[\s\S\r]+?</table>[ \n]*</div>[ \n]*</div>)',data)

    m = m2 + m + m4
    print(target,len(m))
    for test_table in m:
        try:
            soup = BeautifulSoup(test_table,features="lxml")
            table_id = str(soup.find('table').get('id'))

            if table_id == ['team_and_opponent']:
                continue
            soup.findAll('tr')

            table_size = {'on_off':1,'on_off_p':1,'shooting':2,'pbp':1,'playoffs_shooting':2,'playoffs_pbp':1,'contracts':1}

            # use getText()to extract the text we need into a list
            headers = [th.getText() for th in soup.findAll('tr')[table_size.get(table_id,0)].findAll('th')]

            # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
            start_col = 1
            if table_id in ['contracts','injury','on_off','on_off_p','roster']:
                start_col = 0

            headers = headers[start_col:]
            rows = soup.findAll('tr')[start_col:]
            player_stats = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(len(rows))]

            if table_id in ['contracts','injury','on_off','on_off_p','roster']:
                player_names = [[td.getText() for td in rows[i].findAll('th')]
                            for i in range(len(rows))]
                player_stats = [a + b for a,b in zip(player_names[1:],player_stats[1:])]
            headers[0] = 'Name'
            stats = pd.DataFrame(player_stats, columns = headers).set_index('Name')
            if table_id in ['contracts']:
                stats = stats.drop(['Player'])
                stats = stats.iloc[:stats.index.get_loc('')]

            # drop nan
            stats = stats[~ stats.index.isin([None])]
            # convert to float
            obj_cols = stats.loc[:, stats.dtypes == object]
            conv_cols = obj_cols.apply(pd.to_numeric, errors = 'ignore')
            stats.loc[:, stats.dtypes == object] = conv_cols
            
            stats = stats.fillna('')
            
            if True and 'on_off' in table_id:
                stats = stats.iloc[~ stats.index.get_loc('Player')]
                stats = stats.loc[~ (stats.Split == '')]
                stats.index = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in [_ for _ in stats.index if _!='']))

            #print(table_id,stats.index)
            tables[team][table_id]= stats
        except KeyboardInterrupt:
            raise
        except:
            #pass
            print('FAILED TO PARSE ' +str(soup.find('table').get('id') ))

            raise
            

In [ ]:
#rows = soup.findAll('tr')
#player_stats

In [ ]:
import pickle
with open('cbb_{}.pkl'.format(year),'wb') as fp:
    pickle.dump(tables,fp)

In [ ]:
confs = [tables[_]['conf'] for _ in tables]

In [ ]:
len(set(confs))